## Part a - Download the geometries from the source 

In [8]:
import wget

print('Beginning file download with wget module')

url = 'http://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson'
wget.download(url, 'countries.geojson')


Beginning file download with wget module


'countries (1).geojson'

In [4]:
# import geojson
# with open('countries.geojson') as f:
#     gj = geojson.load(f)
# features = gj['features'][0]
# features


{"geometry": {"coordinates": [[[[-69.996938, 12.577582], [-69.936391, 12.531724], [-69.924672, 12.519232], [-69.915761, 12.497016], [-69.880198, 12.453559], [-69.87682, 12.427395], [-69.888092, 12.41767], [-69.908803, 12.417792], [-69.930531, 12.425971], [-69.945139, 12.440375], [-69.924672, 12.440375], [-69.924672, 12.447211], [-69.958567, 12.463202], [-70.027659, 12.522935], [-70.048085, 12.531155], [-70.058095, 12.537177], [-70.062408, 12.54682], [-70.060374, 12.556952], [-70.051096, 12.574042], [-70.048736, 12.583726], [-70.052642, 12.600002], [-70.059641, 12.614244], [-70.061106, 12.625393], [-70.048736, 12.632148], [-70.007151, 12.585517], [-69.996938, 12.577582]]]], "type": "MultiPolygon"}, "properties": {"ADMIN": "Aruba", "ISO_A2": "AW", "ISO_A3": "ABW"}, "type": "Feature"}

## Using ogr2ogr to import geojson file into POSTGIS
Terminal Command:

ogr2ogr -f PostgreSQL PG:"dbname=positiumDB user=enlik" countries.geojson -nln countries -nlt MULTIPOLYGON

and then I export table 'countries' into countries.csv file using 'Postico' Mac OS application

## Read CSV

In [64]:
import pandas as pd

countries = pd.read_csv("countries.csv")

In [65]:
countries.head()

,ogc_fid,admin,iso_a3,iso_a2,wkb_geometry
0,1,Aruba,ABW,AW,0106000020E6100000010000000103000000010000001A...
1,2,Afghanistan,AFG,AF,0106000020E610000001000000010300000001000000FD...
2,3,Angola,AGO,AO,0106000020E6100000030000000103000000010000000E...
3,4,Anguilla,AIA,AI,0106000020E61000000200000001030000000100000018...
4,5,Albania,ALB,AL,0106000020E6100000010000000103000000010000002D...


In [66]:
countries = countries.filter(['ogc_fid', 'wkb_geometry'])
countries

,ogc_fid,wkb_geometry
0,1,0106000020E6100000010000000103000000010000001A...
1,2,0106000020E610000001000000010300000001000000FD...
2,3,0106000020E6100000030000000103000000010000000E...
3,4,0106000020E61000000200000001030000000100000018...
4,5,0106000020E6100000010000000103000000010000002D...
...,...,...
250,251,0106000020E61000000200000001030000000100000037...
251,252,0106000020E61000000900000001030000000100000009...
252,253,0106000020E6100000030000000103000000010000001A...
253,254,0106000020E610000001000000010300000001000000E4...


In [67]:
countries = countries.rename(columns = {"ogc_fid": "geom_id", "wkb_geometry": "geom"})

In [68]:
countries

,geom_id,geom
0,1,0106000020E6100000010000000103000000010000001A...
1,2,0106000020E610000001000000010300000001000000FD...
2,3,0106000020E6100000030000000103000000010000000E...
3,4,0106000020E61000000200000001030000000100000018...
4,5,0106000020E6100000010000000103000000010000002D...
...,...,...
250,251,0106000020E61000000200000001030000000100000037...
251,252,0106000020E61000000900000001030000000100000009...
252,253,0106000020E6100000030000000103000000010000001A...
253,254,0106000020E610000001000000010300000001000000E4...


In [82]:
df_columns = list(countries)
df_columns

columns = ",".join(df_columns)
columns

'geom_id,geom'

In [81]:
values = "VALUES({})".format(",".join(["%s" for _ in df_columns]))
values

'VALUES(%s,%s)'

In [90]:
table = 'test_gis.geom'
table

'test_gis.geom'

In [91]:
import psycopg2
import psycopg2.extras
conn = psycopg2.connect("host=localhost dbname=positiumDB user=enlik")
cur = conn.cursor()

insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)

psycopg2.extras.execute_batch(cur, insert_stmt, countries.values)
conn.commit()

In [92]:
a2_list = pd.read_csv("iso_a2_list_new.csv")
a2_list.head()

,id,iso_a2,country_name_eng,geom_id
0,2,af,Afghanistan,NaN
1,3,ag,Antigua and Barbuda,NaN
2,4,ai,Anguilla,NaN
3,5,al,Albania,NaN
4,6,am,Armenia,NaN


In [93]:
countries

KeyError: 'iso_a2'

In [37]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=positiumDB user=enlik")
cur = conn.cursor()
cur.execute('SELECT * FROM countries')
one = cur.fetchone()
# all = cur.fetchall()
one

(1,
 'Aruba',
 'ABW',
 'AW',
 '0106000020E6100000010000000103000000010000001A000000362D7CD3CD7F51C098543BD7B8272940382D7CD3ED7B51C06810942C3E102940382D7CD32D7B51C0408D3ED7D8092940ACF97BD39B7A51C0707A3DD778FE2840705AD128557851C020A03FD738E82840A8B67CD31D7851C0181DEA81D3DA28406889277ED67851C0408D3ED7D8D5284004D97CD3297A51C030DE3BD7E8D52840342D7CD38D7B51C078673CD718DA28409E28D2287D7C51C0787A3DD778E12840382D7CD32D7B51C0787A3DD778E12840382D7CD32D7B51C0D0C541D7F8E428408AC1D128597D51C0704F42D728ED2840E6E3D128C58151C0005C982CBE0B2940623E7CD3138351C0B055ED81F30F294076A57CD3B78351C0C8163FD708132940AC44277EFE8351C0E8C541D7F8172940A028D228DD8351C0784F42D7281D2940E4E3D128458351C018DE3BD7E8252940B044277E1E8351C078FD922CDE2A2940B044277E5E8351C0E02FEB81333329404206D228D18351C0A8EA912C7E3A2940FE4AD228E98351C0E82FEB8133402940B044277E1E8351C0F0033ED7A8432940725AD128758051C0903C41D7C82B2940362D7CD3CD7F51C098543BD7B8272940')

In [38]:
cur.execute("""
CREATE TABLE users(
id integer PRIMARY KEY,
email text,
name text,
address text)
""")
conn.commit()

In [39]:
update_query = """UPDATE test_gis.iso_a2_list SET geom_id = %s WHERE iso_a2 = %s"""
cur.execute(update_query, (5, 'ae'))
conn.commit()